In [1]:
# enable autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from openai import OpenAI
client = OpenAI()

model = "gpt-4o"
question = "How to be cool?"

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": question}
    ]
)

In [2]:
from askharrison.prompts.query_expansion import generate_search_queries
from askharrison.google_search import run_multiple_google_queries

In [3]:
problem = "I want to find a good restaurant in San Francisco"

search_queries = generate_search_queries(problem, 10, "google")

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [4]:
search_queries

['best San Francisco restaurants',
 'top dining places in San Francisco',
 'popular food joints in San Francisco',
 'highly rated restaurants in San Francisco',
 'San Francisco food critics favorite restaurants',
 'award winning restaurants in San Francisco',
 'top reviewed San Francisco restaurants',
 'San Francisco travel guide best restaurants',
 'best places to eat in San Francisco',
 'Michelin star restaurants in San Francisco']

In [47]:
query_results = run_multiple_google_queries(search_queries)

In [13]:
from askharrison.prompts.content_curation import create_google_reranking_prompt
from askharrison.llm_models import process_question, safe_eval, extract_python_code

In [9]:
google_reranking_prompt = create_google_reranking_prompt(problem, query_results)

In [12]:
reranking_output = process_question(google_reranking_prompt, model="gpt-4o")

In [18]:
python_code = extract_python_code(reranking_output)
llm_output_objs = safe_eval(python_code)

In [22]:
import pandas as pd

pd.DataFrame(llm_output_objs)

,idx,relevance,quality,credibility,recency,overall
0,1,9.5,8.5,9.0,8.0,87.5
1,2,9.0,8.0,9.5,9.0,88.5
2,3,8.5,8.0,9.0,7.0,82.0
3,4,8.0,8.0,8.5,8.5,83.0
4,5,8.0,8.5,9.0,7.5,83.0
5,6,7.5,7.5,8.0,7.0,77.5
6,7,7.0,7.0,9.0,6.5,76.5
7,8,7.5,7.0,9.0,6.0,76.5
8,9,6.5,7.5,8.0,6.5,74.0
9,10,6.0,7.0,8.0,5.5,70.5


In [23]:
import functools
from typing import List, Any, Callable
import tiktoken

In [101]:
def chunk_llm_input(max_tokens: int, encoding_name: str = "cl100k_base"):
    def decorator(func: Callable):
        @functools.wraps(func)
        def wrapper(objects: List[Any], *args, **kwargs):
            encoding = tiktoken.get_encoding(encoding_name)
            
            def get_token_count(obj: Any) -> int:
                return len(encoding.encode(str(obj)))
            
            chunks = []
            current_chunk = []
            current_token_count = 0
            
            for obj in objects:
                if isinstance(obj, str):
                    obj_token_count = get_token_count(obj)
                else:
                    obj_token_count = get_token_count(str(obj))
                print(obj_token_count)
                
                if current_token_count + obj_token_count > max_tokens:
                    chunks.append(current_chunk)
                    current_chunk = [obj]
                    current_token_count = obj_token_count
                else:
                    current_chunk.append(obj)
                    current_token_count += obj_token_count
            
            if current_chunk:
                chunks.append(current_chunk)
            
            results = []
            for chunk in chunks:
                result = func(chunk, *args, **kwargs)
                results.append(result)
            
            return results
        
        return wrapper
    
    return decorator

In [102]:
# Example usage
@chunk_llm_input(max_tokens=3000)
def generate_llm_prompt(objects: List[str]) -> str:
    prompt = "Summarize the following items:\n\n"
    for obj in objects:
        prompt += f"- {obj}\n"
    return prompt

In [112]:
def create_google_reranking_problem(problem: str, max_tokens: int) -> List[str]:
    @chunk_llm_input(max_tokens=max_tokens)
    def helper(query_result: list) -> str:
        return create_google_reranking_prompt(problem, query_result)
    return helper

google_reranking_problem = create_google_reranking_problem(problem, 8000)

In [113]:
flatten_results = []
for query, results in query_results.items():
    items = results['items']
    flatten_results += items

In [114]:
len(flatten_results)

100

In [115]:
reranking_prompt_breakdown = google_reranking_problem(flatten_results)

1653
871
608
1013
904
1139
873
470
1179
2590
1644
1129
603
868
593
866
1168
966
641
1001
1645
642
612
869
603
1131
903
916
1016
867
1649
873
1128
599
1002
1168
872
881
905
1027
1582
941
1732
662
1388
1250
600
1227
1536
1041
1661
610
1650
1022
630
455
614
1130
593
566
1135
1649
872
867
982
1175
655
2586
620
906
883
1714
996
560
956
610
1141
952
973
712
599
1641
591
1013
598
863
943
860
966
916
474
601
1690
903
1019
583
511
869
1588
388


In [116]:
print(reranking_prompt_breakdown[10])

rerank the search results based on the problem statement and criteria below:
    #### problem statement:
    I want to find a good restaurant in San Francisco
    #### criteria:
    - relevance to the problem statement
    - quality of the content
    - credibility of the source
    - recency of the content
    - any other criteria you think is important
    #### search results:
    [{'kind': 'customsearch#result', 'title': "The Ultimate Local's Travel Guide to San Francisco — sarowly | sf + ...", 'htmlTitle': 'The Ultimate Local&#39;s <b>Travel Guide</b> to <b>San Francisco</b> — sarowly | <b>sf</b> + ...', 'link': 'https://sarowly.me/blog/ultimate-locals-travel-guide-to-sanfrancisco', 'displayLink': 'sarowly.me', 'snippet': "Feb 16, 2024 ... In this travel guide, you'll find a list of my favorite places that I love, my go-to restaurants and cafes, as well as best view-points for sunrises and sunset.", 'htmlSnippet': 'Feb 16, 2024 <b>...</b> In this <b>travel guide</b>, you&#39;ll fin

In [121]:
import concurrent.futures
from typing import List, Any, Callable
import time
import random
from tqdm import tqdm

def parallel_llm_processor(prompts: List[str], 
                           llm_function: Callable[[str], Any], 
                           max_workers: int = 5):
    """
    Process a list of LLM prompts in parallel, submitting each prompt individually.
    Displays a progress bar using tqdm.
    
    :param prompts: List of prompts to process
    :param llm_function: Function to call for each prompt
    :param max_workers: Maximum number of parallel workers
    :return: List of results from LLM processing
    """
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all prompts to the executor
        future_to_prompt = {executor.submit(llm_function, prompt): prompt for prompt in prompts}
        
        # Use tqdm to create a progress bar
        with tqdm(total=len(prompts), desc="Processing prompts") as pbar:
            # Collect results as they complete
            for future in concurrent.futures.as_completed(future_to_prompt):
                prompt = future_to_prompt[future]
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    print(f"An error occurred while processing prompt: {prompt[:30]}...")
                    print(f"Error: {str(e)}")
                finally:
                    pbar.update(1)  # Update the progress bar
    
    return results

In [122]:
help(process_question)

Help on function process_question in module askharrison.llm_models:

process_question(question, model='gpt-4o')



In [123]:
len(reranking_prompt_breakdown)

14

In [124]:
reranking_llm_responses = parallel_llm_processor(reranking_prompt_breakdown, process_question)

Processing prompts: 100%|██████████| 14/14 [00:21<00:00,  1.57s/it]


In [131]:
eval_reranking_responese =  [safe_eval(extract_python_code(response)) for response in reranking_llm_responses]

flatten_eval_reranking_responese = []
for response in eval_reranking_responese:
    flatten_eval_reranking_responese += response

In [133]:
pd.DataFrame(flatten_eval_reranking_responese)

,idx,relevance,quality,credibility,recency,overall
0,0,9.0,8.5,9.0,9.0,88.5
1,1,9.5,9.0,9.5,9.0,91.5
2,2,8.5,8.0,8.5,8.5,84.5
3,3,8.0,8.0,9.0,7.0,82.0
4,4,7.0,7.0,6.5,8.5,75.5
...,...,...,...,...,...,...
94,5,8.5,8.0,8.0,8.0,84.5
95,7,8.0,7.5,7.5,6.0,77.0
96,8,8.0,7.0,7.0,7.0,76.0
97,6,7.5,7.0,7.0,6.0,73.5


In [129]:
eval_reranking_responese

[[{'idx': 0,
   'relevance': 9.0,
   'quality': 8.5,
   'credibility': 9.0,
   'recency': 9.0,
   'overall': 88.5},
  {'idx': 1,
   'relevance': 9.5,
   'quality': 9.0,
   'credibility': 9.5,
   'recency': 9.0,
   'overall': 91.5},
  {'idx': 2,
   'relevance': 8.5,
   'quality': 8.0,
   'credibility': 8.5,
   'recency': 8.5,
   'overall': 84.5},
  {'idx': 3,
   'relevance': 8.0,
   'quality': 8.0,
   'credibility': 9.0,
   'recency': 7.0,
   'overall': 82.0},
  {'idx': 4,
   'relevance': 7.0,
   'quality': 7.0,
   'credibility': 6.5,
   'recency': 8.5,
   'overall': 75.5}],
 [{'idx': 8,
   'relevance': 10.0,
   'quality': 9.0,
   'credibility': 10.0,
   'recency': 10.0,
   'overall': 97.0},
  {'idx': 2,
   'relevance': 9.0,
   'quality': 8.0,
   'credibility': 8.0,
   'recency': 9.0,
   'overall': 85.0},
  {'idx': 4,
   'relevance': 9.0,
   'quality': 8.0,
   'credibility': 9.0,
   'recency': 8.0,
   'overall': 84.0},
  {'idx': 6,
   'relevance': 8.0,
   'quality': 8.0,
   'credibility